In [342]:
import requests
import pandas as pd
import time

#TMDb api key
api_key = "" #send me an email, happy to share the API key for testing

In [343]:
#does 'discover' query on tmdb API, returns page N of the most popular American movies (20 results per page)
def popular_page(page_num):
    response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key={}&popularity.desc\
                          &include_adult=false&include_video=false&language=en-US&page={}'.format(api_key, page_num))
    print("{} for page {}".format(response, page_num))
    return response.json()['results']

#test = popular_page(1)

In [344]:
#extract results from the movie page results, append each movie's details to their respective arrays
#'title','id','vote_average', 'vote_count', 'popularity', 'release_year'
#later, through separate API calls: 'budget,'revenue','director name', 'gender'
titles = []
ids = []
vote_average = []
vote_count = []
popularity = []
release_year = []

In [345]:
#function to go through movies in a results page and append the attributes to the arrays
def discover_result_to_arrays(results_page):
   for index, movie in enumerate(results_page):
       titles.append(results_page[index]['title'])
       ids.append(results_page[index]['id'])
       vote_average.append(results_page[index]['vote_average'])
       vote_count.append(results_page[index]['vote_count'])
       popularity.append(results_page[index]['popularity'])
       release_year.append(results_page[index]['release_date'][:4]) #gets the year from the yyyy-mm-dd date
   return

#discover_result_to_arrays(test)
#release_year

In [346]:
#loop through the discover queries for a given range of pages

def discover_pulls(start_page, end_page):
    for page in range(start_page, end_page + 1):
        discover_result = popular_page(page) #does discover query on the next page
        discover_result_to_arrays(discover_result) #appends the query results to the result arrays

#stop at 50 pages (1000 movies), because Genderize API is limited to 1000 calls per day
discover_pulls(1,25)
time.sleep(10)
discover_pulls(26,50)

<Response [200]> for page 51
<Response [200]> for page 52
<Response [200]> for page 53
<Response [200]> for page 54
<Response [200]> for page 55
<Response [200]> for page 56
<Response [200]> for page 57
<Response [200]> for page 58
<Response [200]> for page 59
<Response [200]> for page 60
<Response [200]> for page 61
<Response [200]> for page 62
<Response [200]> for page 63
<Response [200]> for page 64
<Response [200]> for page 65
<Response [200]> for page 66
<Response [200]> for page 67
<Response [200]> for page 68
<Response [200]> for page 69
<Response [200]> for page 70
<Response [200]> for page 71
<Response [200]> for page 72
<Response [200]> for page 73
<Response [200]> for page 74
<Response [200]> for page 75
<Response [200]> for page 76
<Response [200]> for page 77
<Response [200]> for page 78
<Response [200]> for page 79
<Response [200]> for page 80
<Response [200]> for page 81
<Response [200]> for page 82
<Response [200]> for page 83
<Response [200]> for page 84
<Response [200

In [348]:
# pull details for a movie, including cast in the response
# more details here: https://www.themoviedb.org/talk/56f6d2dc9251413007001c3c
# &append_to_response=credits

def movie_details_w_cast(movie_id):
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}&append_to_response=credits'
                            .format(movie_id, api_key))
    return response.json()

# extract director's name from the JSON for the movie details
# return as soon as the director is found, to save on iterations
def get_director(movie):
    for person in movie['credits']['crew']:
        if person['job'] == 'Director':
            director = person['name']
            return director

# extract revenue and budget from the movie details, append to their respective arrays
# append result from get_director to its array
def detail_results_to_arrays(movie):
    revenues.append(movie['revenue'])
    budgets.append(movie['budget'])
    
    director = get_director(movie)
    directors.append(director)
    
    return

In [349]:
revenues = []
budgets = []
directors = []

In [351]:
# this cell takes a while - go grab a snack!

# get detailed results for all of the movie IDs pulled in the discover search
# add pause after each 40 to avoid API rate limiting (40 calls/10 secs)
rate_limiter = 0

for _id in ids:
    movie = movie_details_w_cast(_id)
    detail_results_to_arrays(movie)
    
    rate_limiter += 1
    if rate_limiter == 40:
        time.sleep(10)
        rate_limiter = 0

In [352]:
# get genders for every director
# keep track of Genderize API calls - limited to 1000 per day
genders = []
genderize_api_call_count = 0

def get_gender(director):
    try:
        director_first_name = director.split()[0] #get the first name
        director_gender_api_call = requests.get('https://api.genderize.io/?name={}'.format(director_first_name))
        json = director_gender_api_call.json()
        
        #don't make overconfident guesses about gender
        if 'probability' in json and json['probability'] < .6:
            return None
        
        #if genderize is confident about gender, return the gender
        else:
            gender = json['gender']
            return gender
        
    except AttributeError:
        return None

for director in directors[0:1000]:
    gender = get_gender(director)
    genders.append(gender)
    genderize_api_call_count += 1

In [353]:
genderize_api_call_count

1000

In [354]:
#make a dataframe to store the data
df_limiter = len(genders) #the limiting factor is the amount of data pulled from the genderize API

df = pd.DataFrame({'Title':titles[:df_limiter],
                   'ID':ids[:df_limiter],
                   'Vote_Avg':vote_average[:df_limiter],
                   'Vote_Count':vote_count[:df_limiter],
                   'Popularity':popularity[:df_limiter],
                   'release_year':release_year[:df_limiter],
                   'Director':directors[:df_limiter],
                   'Gender':genders,
                   'Revenue':revenues[:df_limiter],
                   'Budget':budgets[:df_limiter]
                  })

df['Gender'].value_counts()

df.to_csv('Movie data up to row 2000.csv')